In [67]:
import pandas as pd

In [68]:
# read in dataset
plane_df = pd.read_csv("../Data/Aviation_data.csv")

/var/folders/gk/hcbv44_53gj8f9w4481whvbw0000gn/T/ipykernel_43046/605794956.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  plane_df = pd.read_csv("../Data/Aviation_data.csv")


In [69]:
## Creating new df
# Dropping columns that we dont want
# filtering dataframe
# Saving new df to csv

new_df = plane_df.drop(columns=['Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name',
       'Amateur.Built', 'Schedule', 'Air.carrier', 'Report.Status',
       'Publication.Date'])
new_df = new_df.dropna(subset=['FAR.Description']).query('`FAR.Description` != ["Part 91: General Aviation", "091"]')
new_df = new_df[new_df['FAR.Description'].isin(far_cols)]
new_df = new_df[new_df['Aircraft.damage'] != 'Unknown']
new_df = new_df.drop(columns=["Purpose.of.flight"])
new_df[spec_cols] = new_df[spec_cols].fillna(0)
new_df['Make'] = new_df['Make'].str.title()
new_df = new_df[new_df['Aircraft.Category'] == 'Airplane']
new_df.to_csv('new_df.csv', index=False)

In [61]:
## reading in big plane csv
big_plane_yiyi = pd.read_csv("../Data/big_plane.csv")

In [62]:
## Creating severity score
# adding new column for severity score
# resave df with new column
numerator = big_plane_yiyi['Total.Uninjured'] + 3 * (big_plane_yiyi["Total.Minor.Injuries"]) + 12 * (big_plane_yiyi["Total.Serious.Injuries"]) + 25 * (big_plane_yiyi["Total.Fatal.Injuries"])
molecular = big_plane_yiyi['People.Count']
big_plane_yiyi['Severity_Score'] = (numerator / molecular) * 100
big_plane_yiyi = big_plane_yiyi[['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured','Make:Model','Severity_Score']]

# creating function the total.fatalities value for new Fatality column
def format_fatality(value):
    if value == 0.0:
        return 'Non-Fatal'
    else:
        return f'Fatal-{int(value)}'

# using function Fatality column
big_plane_yiyi['Fatality'] = big_plane_yiyi['Total.Fatal.Injuries'].apply(format_fatality)

# splitting make:model for make and model standardization
big_plane_yiyi['extracted_value'] = big_plane_yiyi['Make:Model'].str.split(':', expand=True)[0]

# mcdonnel douglas cleaning and creating subset
mcdonnell_douglas_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Mcdonnell Douglas')]
mcdonnell_douglas_df = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Mcdonnell Douglas')].groupby('Make:Model')
mcdonnell_douglas_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Mcdonnell Douglas')]
mcdonnell_douglas_filtered['Make:Model'] = mcdonnell_douglas_filtered['Make:Model'].str.replace("-", "", regex=False).str.replace(" ", "")

# more name cleaning
def normalize_model(name):
    if 'DC983' in name or 'DC983(MD83)' in name or 'MD83' in name or 'MD88' in name or 'MD80' in name or 'MD83' in name or 'DC982' in name or 'MD81' in name or 'DC982' in name:
        return 'McdonnellDouglas:MD-80'
    elif 'DC1030' in name or 'DC1010' in name or 'DC10' in name or 'DC1030F' in name:
        return 'McdonnellDouglas:DC-10'
    elif 'DC9' in name:
        return 'McdonnellDouglas:DC-9'
    elif 'MD90' in name:
        return 'McdonnellDouglas:MD-90'
    elif 'MD11' in name:
        return 'McdonnellDouglas:MD-11'
    elif 'DC8' in name:
        return 'McdonnellDouglas:DC-8'
    else:
        return name

mcdonnell_douglas_filtered['Make:Model'] = mcdonnell_douglas_filtered['Make:Model'].apply(normalize_model)

Boeing_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Boeing')]
Boeing_filtered['Make:Model'] = Boeing_filtered['Make:Model'].apply(lambda x: x.split(':')[0] if 'Boeing' in x else '') + \
                  Boeing_filtered['Make:Model'].str.extract('(\d+)')[0]

Airbus_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Airbus')]
Airbus_filtered['Make:Model'] = Airbus_filtered['Make:Model'].apply(lambda x: x.split(':')[0] if 'Airbus' in x else '') + \
                  Airbus_filtered['Make:Model'].str.extract('(\d+)')[0]

# more name cleaning
Embraer_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Embraer')]
def normalize_model(name):
    if 'EMB145' in name or 'EMB-145LR' in name or 'EMB 145LR' in name or 'EMB-145' in name or 'Embraer: 145' in name:
        return 'Embraer-EMB145'
    elif 'ERJ170' in name or 'ERJ 170' in name:
        return 'Embraer-ERJ170'
    elif 'ERJ190' in name or '190' in name:
        return 'Embraer-ERJ190'
    elif 'EMB-135' in name or '135KL' in name:
        return 'Embraer-EMB135KL'
    elif 'EMB-170' in name or 'EMB170' in name:
        return 'Embraer-EMB170'
    elif 'E175' in name:
        return 'Embraer-E175'
    elif 'ERJ175' in name:
        return 'Embraer-ERJ175'
    else:
        return name
    
Embraer_filtered['Make:Model'] = Embraer_filtered['Make:Model'].apply(normalize_model)

# creating subset with cleaned Bombardier names
Bombardier_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Bombardier')]
def normalize_model(name):
    if '600' in name:
        return 'Bombardier-CL600'
    elif '402' in name:
        return 'Bombardier-DHC8-402'
    elif '202' in name:
        return 'Bombardier-DHC8-202'
    elif '311' in name:
        return 'Bombardier-DHC8-311'
    else:
        return name

# Applying the function to the 'Make:Model' column
Bombardier_filtered['Make:Model'] = Bombardier_filtered['Make:Model'].apply(normalize_model)

# last time
others_filtered = big_plane_yiyi[big_plane_yiyi['Make:Model'].str.contains('Lockheed|Fokker|Atr|Dehavilland|Short Brothers|Saab|Bae')]
def normalize_model(name):
    if 'L-1011' in name:
        return 'Lockheed-L-1011'
    elif '4000' in name:
        return 'Fokker-F28-4000'
    elif 'DHC8' in name:
        return 'Dehavilland-DHC8'
    elif '100' in name:
        return 'Fokker-F-100'
    else:
        return name

others_filtered['Make:Model'] = others_filtered['Make:Model'].apply(normalize_model)

# taking subsets of cleaned names and combining using concat
combined_big_plane_df = pd.concat([Boeing_filtered, Bombardier_filtered, mcdonnell_douglas_filtered, others_filtered, Airbus_filtered, Embraer_filtered], axis=0, ignore_index=True)
combined_big_plane_df.to_csv('combined_big_plane_df.csv', index=False)

# sorting based on severity score
sorted_df = combined_big_plane_df.sort_values(by='Severity_Score', ascending=True)
event_counts = sorted_df['Make:Model'].value_counts()
normalized_severity_scores = sorted_df.groupby('Make:Model')['Severity_Score'].sum() / event_counts


/var/folders/gk/hcbv44_53gj8f9w4481whvbw0000gn/T/ipykernel_43046/1726905385.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcdonnell_douglas_filtered['Make:Model'] = mcdonnell_douglas_filtered['Make:Model'].str.replace("-", "", regex=False).str.replace(" ", "")
/var/folders/gk/hcbv44_53gj8f9w4481whvbw0000gn/T/ipykernel_43046/1726905385.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcdonnell_douglas_filtered['Make:Model'] = mcdonnell_douglas_filtered['Make:Model'].apply(normalize_model)
/var/fol

In [64]:
# Loading in final_df
df = pd.read_csv('../Data/final_df.csv')
# Getting total deliveries from sources and adding to dict
total_deliveries = {
    'Boeing737': 11550,
    'Boeing777': 1718,
    'Boeing767': 1288,
    'Boeing757': 1049,
    'Airbus320': 11134,
    'McdonnellDouglas:MD-80': 1191,
    'Bombardier-CL600': 1157,
    'Embraer-EMB145': 1231,
    'Airbus319': 1500,
    'Boeing747': 1574,
    'Airbus330': 1579,
    'Airbus321': 2965,
    'Embraer-ERJ170': 191,
    'McdonnellDouglas:DC-9': 2441,
    'McdonnellDouglas:DC-10': 386,
    'Boeing727': 1831,
    'Boeing787': 1040,
    'Embraer-ERJ190': 463,
    'Lockheed-L-1011': 250,
    'Bombardier-DHC8-402':1249,
    'Boeing717': 156,
    'Embraer-EMB135KL': 892,
    'McdonnellDouglas:MD-11': 200,
    'McdonnellDouglas:MD-90': 116,
    'Airbus220': 296,
    'Airbus310': 255,
    'Embraer-EMB170': 160,
    'Atr: ATR72': 1800,
    'Dehavilland-DHC8': 1232,
    'Airbus380': 251,
    'Fokker-F-100': 278,
    'Short Brothers: SD3-60': 165,
    'Boeing82': 1190,
    'McdonnellDouglas:DC-8': 556,
    'Boeing707': 1011,
    'Fokker-F28-4000': 241,
    'Boeing10': 400,
    'Airbus340': 377,
    'Embraer-ERJ175': 600,
    'Boeing88': 400,
    'Airbus300': 816,
    'Bombardier-DHC8-202': 500,
    'Boeing9': 500,
    'Boeing90': 450,
    'Boeing744': 1574,
    'Saab: 2000': 63,
}

# creating new column and mapping total_deliveries dict to make:model column
df['Total Deliveries'] = df['Make:Model'].map(total_deliveries)

In [65]:
# adjusted the severity score based on the volume of planes that are active
df['Adjusted Severity_Score'] = (df['Severity_Score'] / df['Total Deliveries']) * 250

# creating new df that is grouped by make:model for clean view of adj severity scores
grouped_df = df.groupby('Make:Model').agg({
    'Adjusted Severity_Score': 'mean',
    'Total Deliveries': 'mean',
    'Total.Fatal.Injuries': 'sum',
    'Total.Serious.Injuries': 'sum',
    'Total.Minor.Injuries': 'sum',
    'Total.Uninjured': 'sum',
    'Make:Model': 'size'
})

# ranking df
Sorted_df = grouped_df.sort_values('Adjusted Severity_Score', ascending=True)

Sorted_df.rename(columns={'Make:Model': 'Number of Events'}, inplace=True)

filepath = '../Data/plane_adj_severity.csv'
Sorted_df.to_csv(filepath, index=True)

In [66]:
# see top 5
Sorted_df.head()

,Adjusted Severity_Score,Total Deliveries,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Number of Events
Make:Model,,,,,,,
Airbus320,3.670950,11134.0,100,53,21,7112,45
Boeing737,4.907782,11550.0,1314,683,189,30499,236
McdonnellDouglas:DC-9,11.843467,2441.0,3,4,55,1277,15
Atr: ATR72,13.888889,1800.0,0,0,0,142,3
Boeing744,15.883100,1574.0,0,0,0,159,1
Dehavilland-DHC8,20.292208,1232.0,0,0,0,104,2
Embraer-EMB145,21.029687,1231.0,0,3,8,2119,30
Airbus319,21.131053,1500.0,0,68,27,3359,27
Airbus321,21.167247,2965.0,157,1,4,2575,16
